In [41]:
# 1
import math
import pandas as pd

def f(x):
    return x**3 + 2*x**2 - 3*x - 1

def f_prime(x):
    return 3*x**2 + 4*x - 3

def g(x):
    return math.sqrt((3*x + 1) / (x + 2))

def bisection(a, b, n):
    e = []
    for _ in range(n):
        x = (a + b) / 2
        e.append(abs(f(x)))
        if f(a) * f(x) < 0:
            b = x
        else:
            a = x
    return x, e

def secant(x0, x1, n):
    e = []
    for _ in range(n):
        f0, f1 = f(x0), f(x1)
        if abs(f1 - f0) < 1e-15:
            break
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0)
        e.append(abs(f(x2)))
        x0, x1 = x1, x2
    return x1, e

def fixed_point(x0, n):
    e = []
    for _ in range(n):
        x1 = g(x0)
        e.append(abs(f(x1)))
        x0 = x1
    return x0, e

def newton(x0, n):
    e = []
    for _ in range(n):
        f0, fp0 = f(x0), f_prime(x0)
        if abs(f_prime(x0)) < 1e-15:
            break
        x1 = x0 - f0 / fp0
        e.append(abs(f(x1)))
        x0 = x1
    return x0, e
    
def false_position(a, b, n):
    e = []
    for _ in range(n):
        fa, fb = f(a), f(b)
        if abs(fb - fa) < 1e-15: 
            break
        c = b - fb * (b - a) / (fb - fa)
        e.append(abs(f(c)))
        if fa * f(c) < 0:
            b = c
        else:
            a = c
    return c, e
    
x0, x1 = 1.0, 2.0
n = 20

r1, e1 = bisection(x0, x1, n)
r2, e2 = secant(x0, x1, n)
r3, e3 = fixed_point(x0, n)
r4, e4 = newton(x0, n)
r5, e5 = false_position(x0, x1, n)

df1 = pd.DataFrame({
    'Method': ['Bisection', 'Secant', 'Fixed-Point', 'Newton-Raphson', 'False Position'],
    'Approximation Root': [r1, r2, r3, r4, r5]
})
print(df1.round(5).to_markdown(index=False))

print("\n Error at Each Step\n")

def pad(e, n):
    return e[:n] if len(e) >= n else e + [e[-1]] * (n - len(e))

df2 = pd.DataFrame({
    'n': range(1, n + 1),
    'Bisection': pad(e1, n),
    'Secant': pad(e2, n),
    'Fixed-Point': pad(e3, n),
    'Newton-Raphson': pad(e4, n),
    'False Position': pad(e5, n)
})
for col in df2.columns[1:]:
    df2[col] = df2[col].map('{:.2e}'.format)
print(df2.round(5).to_markdown(index=False))

| Method         |   Approximation Root |
|:---------------|---------------------:|
| Bisection      |              1.19869 |
| Secant         |              1.19869 |
| Fixed-Point    |              1.19869 |
| Newton-Raphson |              1.19869 |
| False Position |              1.19869 |

 Error at Each Step

|   n |   Bisection |   Secant |   Fixed-Point |   Newton-Raphson |   False Position |
|----:|------------:|---------:|--------------:|-----------------:|-----------------:|
|   1 |    2.38     | 0.549    |      0.258    |         0.328    |         0.549    |
|   2 |    0.328    | 0.274    |      0.0553   |         0.0137   |         0.274    |
|   3 |    0.42     | 0.0292   |      0.0114   |         2.81e-05 |         0.131    |
|   4 |    0.0676   | 0.00129  |      0.00232  |         1.18e-10 |         0.0609   |
|   5 |    0.125    | 5.61e-06 |      0.000474 |         0        |         0.028    |
|   6 |    0.0272   | 1.09e-09 |      9.66e-05 |         0        |        

In [45]:
# 2
def f(x):
    return x**3 + 2*x**2 - 3*x - 1
    
def f_prime(x):
    return 3*x**2 + 4*x - 3

def f_double_prime(x):
    return 6*x + 4

TOL = 1.0e-15 
def chebyshev(x0, max_iter):
    errors = []
    for n in range(max_iter):
        f_n = f(x0)
        f_p_n = f_prime(x0)
        f_pp_n = f_double_prime(x0)

        if abs(f_p_n) < TOL:
            break

        u_n = f_n / f_p_n
        x_new = x0 - u_n - 0.5 * (f_pp_n / f_p_n) * (u_n**2)

        error = abs(f(x_new))  
        errors.append(error)
        
        if error < TOL:
            break
        x0 = x_new

    while len(errors) < max_iter:
        errors.append(errors[-1])  
    return x0, errors

x0 , x1 ,max_iterations = 1.0, 2.0 , 20

r6, err_cheby = chebyshev(x0, max_iterations)

df_final = pd.DataFrame({
    'Method': ['Bisection', 'Secant', 'Fixed-Point', 'Newton-Raphson', 'False Position' , 'Chebyshev'],
    'Approximation Root': [r1, r2, r3, r4, r5 , r6]
})

print("\nApproximate Root (at 20 Iterations) - Comparison\n")
print(df_final.to_markdown(index=False))

def pad(e, n):
    return e[:n] if len(e) >= n else e + [e[-1]] * (n - len(e))

df_error = pd.DataFrame({
    'n': range(1, n + 1),
    'Bisection': pad(e1, n),
    'Secant': pad(e2, n),
    'Fixed-Point': pad(e3, n),
    'Newton-Raphson': pad(e4, n),
    'False Position': pad(e5, n),
    'Chebyshev': pad(err_cheby, max_iterations)
})

for col in df_error.columns[1:]:
    df_error[col] = df_error[col].map('{:.2e}'.format)

print("\n Error at Each Step\n")
print(df_error.to_markdown(index=False))


Approximate Root (at 20 Iterations) - Comparison

| Method         |   Approximation Root |
|:---------------|---------------------:|
| Bisection      |              1.19869 |
| Secant         |              1.19869 |
| Fixed-Point    |              1.19869 |
| Newton-Raphson |              1.19869 |
| False Position |              1.19869 |
| Chebyshev      |              1.19869 |

 Error at Each Step

|   n |   Bisection |   Secant |   Fixed-Point |   Newton-Raphson |   False Position |   Chebyshev |
|----:|------------:|---------:|--------------:|-----------------:|-----------------:|------------:|
|   1 |    2.38     | 0.549    |      0.258    |         0.328    |         0.549    |    0.16     |
|   2 |    0.328    | 0.274    |      0.0553   |         0.0137   |         0.274    |    0.000196 |
|   3 |    0.42     | 0.0292   |      0.0114   |         2.81e-05 |         0.131    |    3.06e-13 |
|   4 |    0.0676   | 0.00129  |      0.00232  |         1.18e-10 |         0.0609   |

In [13]:
'''
Observations:
Fastest Convergence:
                The Chebyshev method is the clear winner.
                It achieved full machine precision (error 3.06e-13) in only 3 iterations.
                Chebyshev vs.Newton-Raphson:Chebyshev's cubic (order 3) convergence is visibly faster than 
                --Newton's quadratic (order 2) convergence.
                Iteration 2: Newton's error was 0.0137. Chebyshev's error was already 0.000196.
                Iteration 3: Newton's error was 2.810e-05. 
                Chebyshev's error was 3.06e-13, effectively zero.Chebyshev converged in 3 steps, while Newton required 4.

Overall Ranking (Speed):
                     Chebyshev (Cubic): 3 iterations
                     Newton-Raphson (Quadratic): 4 iterations
                     Secant (Superlinear, ~1.618): 7 iterations
                     Fixed-Point (Linear ): ~19 iterations
                     Bisection (Linear ): >20 iterations
                     False Position (Linear, slow ): >20 iterations
                     
            This incredible speed comes at the cost of calculating the second derivative, f''(x) = 6x + 4.
            For this simple polynomial, it's easy. For a very complex function, calculating f''(x) might be more 
            computationally expensive than the extra iteration Newton's method would take.
'''

In [2]:
# 3
import math
import pandas as pd

def f(x): 
    return x**3 - 9*x + 1
def f_prime(x): 
    return 3*x**2 - 9
def f_double_prime(x): 
    return 6*x

def bisection(a, b, n):
    e = []
    for _ in range(n):
        x = (a + b) / 2
        e.append(abs(f(x)))
        if f(a) * f(x) < 0:
            b = x
        else:
            a = x
    return x, e

def secant(x0, x1, n):
    e = []
    for _ in range(n):
        f0, f1 = f(x0), f(x1)
        if abs(f1 - f0) < 1e-15:
            break
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0)
        e.append(abs(f(x2)))
        x0, x1 = x1, x2
    return x1, e

def fixed_point(x0, n):
    def g(x): return (9*x - 1)**(1/3)
    e = []
    for _ in range(n):
        x1 = g(x0)
        e.append(abs(f(x1)))
        x0 = x1
    return x0, e

def newton(x0, n):
    e = []
    for _ in range(n):
        f0, fp0 = f(x0), f_prime(x0)
        if abs(fp0) < 1e-15: break
        x1 = x0 - f0/ fp0
        e.append(abs(f(x1)))
        x0 = x1
    return x0, e

def false_position(a, b, n):
    e = []
    for _ in range(n):
        fa, fb = f(a), f(b)
        if abs(fb - fa) < 1e-15: break
        c = b - fb * (b - a) / (fb - fa)
        e.append(abs(f(c)))
        if fa * f(c) < 0:
            b = c
        else:
            a = c
    return c, e

def chebyshev(x0, n, tol=1e-15):
    errors = []
    for _ in range(n):
        f_n = f(x0); f_p_n = f_prime(x0); f_pp_n = f_double_prime(x0)
        if abs(f_p_n) < 1e-15: break
        u_n = f_n / f_p_n
        x_new = x0 - u_n - 0.5 * (f_pp_n / f_p_n) * (u_n**2)
        error = abs(f(x_new))
        errors.append(error)
        if error < tol or abs(x_new - x0) < 1e-12:
            x0 = x_new
            break
        x0 = x_new
    if len(errors) == 0:
        errors = [abs(f(x0))] * n
    while len(errors) < n:
        errors.append(errors[-1])
    return x0, errors

a, b ,n = 2.0, 3.0, 20

r1, e1 = bisection(a, b, n); r2, e2 = secant(a, b, n) ;r3, e3 = fixed_point(a, n)
r4, e4 = newton((a+b)/2, n); r5, e5 = false_position(a, b, n); r6, e6 = chebyshev((a+b)/2 , n)

df1 = pd.DataFrame({
    'Method': ['Bisection', 'Secant', 'Fixed-Point', 'Newton-Raphson', 'False Position', 'Chebyshev'],
    'Approximation Root': [r1, r2, r3, r4, r5, r6]
})
print(df1.round(5).to_markdown(index=False))

print("\n Error at Each Step\n")

def pad(e, n):
    return e[:n] if len(e) >= n else e + [e[-1]] * (n - len(e))

df2 = pd.DataFrame({
    'n': range(1, n + 1),
    'Bisection': pad(e1, n),
    'Secant': pad(e2, n),
    'Fixed-Point': pad(e3, n),
    'Newton-Raphson': pad(e4, n),
    'False Position': pad(e5, n),
    'Chebyshev': pad(e6, n)
})
print(df2.round(5).to_markdown(index=False))


| Method         |   Approximation Root |
|:---------------|---------------------:|
| Bisection      |              2.94282 |
| Secant         |              2.94282 |
| Fixed-Point    |              2.94282 |
| Newton-Raphson |              2.94282 |
| False Position |              2.94282 |
| Chebyshev      |              2.94282 |

 Error at Each Step

|   n |   Bisection |   Secant |   Fixed-Point |   Newton-Raphson |   False Position |   Chebyshev |
|----:|------------:|---------:|--------------:|-----------------:|-----------------:|------------:|
|   1 |     5.875   |  0.711   |       5.14153 |          2.94191 |          0.711   |     1.90557 |
|   2 |     2.95312 |  0.02147 |       2.13078 |          0.20063 |          0.02147 |     0.01768 |
|   3 |     1.11133 |  0.00049 |       0.78599 |          0.00121 |          0.00062 |     0       |
|   4 |     0.09009 |  0       |       0.27829 |          0       |          2e-05   |     0       |
|   5 |     0.44626 |  0       |    

In [ ]:
'''
Observations:
Fastest Convergence:
                The Chebyshev method is the clear winner.
                It achieved full machine precision in only 2 iterations.
                Chebyshev vs.Newton-Raphson:Chebyshev's cubic (order 3) convergence is visibly faster than 
                --Newton's quadratic (order 2) convergence.
                Iteration 2: Newton's error was 0.20063. Chebyshev's error was already 0.01768.
                Iteration 3: Newton's error was 0.00121 
                Chebyshev's error was effectively zero.Chebyshev converged in 2 steps, while Newton required 3.
            
Overall Ranking (Speed):
                     Chebyshev (Cubic): 2 iterations
                     Newton-Raphson (Quadratic): 3 iterations
                     Secant (Superlinear, ~1.618): 3 iterations
                     Fixed-Point (Linear ): ~14 iterations
                     Bisection (Linear ): >20 iterations
                     False Position (Linear, slow ): ~4 iterations

            This incredible speed comes at the cost of calculating the second derivative, f''(x) = 6*x.
            For this simple polynomial, it's easy. For a very complex function, calculating f''(x) might be more 
            computationally expensive than the extra iteration Newton's method would take.
'''